In [1]:
%load_ext lab_black

In [2]:
from pyspark.sql import SparkSession, functions as F
import os
import sys

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

# set the total amount of memory to 24gb
spark = (
    SparkSession.builder.config("spark.driver.memory", "6g")
    .config("spark.driver.cores", 8)
    .config("spark.sql.shuffle.partitions", 64)
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .getOrCreate()
)
spark

In [3]:
! cat ../data/raw/2022-12-10-mangadex-manga.ndjson | head -n 1

{"id": "84af2e9d-2838-40d0-b796-3c08f18e806b", "type": "manga", "attributes": {"title": {"en": "Shibaraku"}, "altTitles": [{"ja": "\u30b7\u30d0\u30e9\u30af"}, {"it": "Shiba Raku"}], "description": {}, "isLocked": false, "links": {"al": "76226", "ap": "shibaraku", "kt": "shibaraku", "mu": "48281", "mal": "46226"}, "originalLanguage": "ja", "lastVolume": "2", "lastChapter": "", "publicationDemographic": null, "status": "completed", "year": 2009, "contentRating": "suggestive", "tags": [], "state": "published", "chapterNumbersResetOnNewVolume": false, "createdAt": "2022-08-31T11:14:19+00:00", "updatedAt": "2022-09-28T19:20:29+00:00", "version": 4, "availableTranslatedLanguages": [], "latestUploadedChapter": null}, "relationships": [{"id": "4bba09c3-40c4-4cd6-9089-c47f73d5e1eb", "type": "author"}, {"id": "4bba09c3-40c4-4cd6-9089-c47f73d5e1eb", "type": "artist"}, {"id": "6e663bb1-cade-4868-a0fd-fdee1768901f", "type": "cover_art"}]}


In [4]:
raw = spark.read.text("../data/raw/2022-12-10-mangadex-manga.ndjson")
raw.show(truncate=80, n=3)

+--------------------------------------------------------------------------------+
|                                                                           value|
+--------------------------------------------------------------------------------+
|{"id": "84af2e9d-2838-40d0-b796-3c08f18e806b", "type": "manga", "attributes":...|
|{"id": "84b659e3-7408-462d-bb35-8aeb6ed427a1", "type": "manga", "attributes":...|
|{"id": "84c7d300-80fc-4f31-99da-5833ebdc842a", "type": "manga", "attributes":...|
+--------------------------------------------------------------------------------+
only showing top 3 rows



In [5]:
raw.limit(1).rdd.map(lambda r: r.value).collect()

['{"id": "84af2e9d-2838-40d0-b796-3c08f18e806b", "type": "manga", "attributes": {"title": {"en": "Shibaraku"}, "altTitles": [{"ja": "\\u30b7\\u30d0\\u30e9\\u30af"}, {"it": "Shiba Raku"}], "description": {}, "isLocked": false, "links": {"al": "76226", "ap": "shibaraku", "kt": "shibaraku", "mu": "48281", "mal": "46226"}, "originalLanguage": "ja", "lastVolume": "2", "lastChapter": "", "publicationDemographic": null, "status": "completed", "year": 2009, "contentRating": "suggestive", "tags": [], "state": "published", "chapterNumbersResetOnNewVolume": false, "createdAt": "2022-08-31T11:14:19+00:00", "updatedAt": "2022-09-28T19:20:29+00:00", "version": 4, "availableTranslatedLanguages": [], "latestUploadedChapter": null}, "relationships": [{"id": "4bba09c3-40c4-4cd6-9089-c47f73d5e1eb", "type": "author"}, {"id": "4bba09c3-40c4-4cd6-9089-c47f73d5e1eb", "type": "artist"}, {"id": "6e663bb1-cade-4868-a0fd-fdee1768901f", "type": "cover_art"}]}']

In [6]:
test = spark.read.json(raw.limit(10).rdd.map(lambda r: r.value))
test.show()
test.printSchema()

+----------------------+--------------------+--------------------+-----+
|            attributes|                  id|       relationships| type|
+----------------------+--------------------+--------------------+-----+
|{[{null, null, シバ...|84af2e9d-2838-40d...|[{4bba09c3-40c4-4...|manga|
|  {[{Imperfect Love...|84b659e3-7408-462...|[{085a74c6-7ba4-4...|manga|
|  {[{Never LAN Chro...|84c7d300-80fc-4f3...|[{0b365f12-8342-4...|manga|
|  {[], [en], false,...|84d1ca67-ba97-45b...|[{7d3f812a-70c6-4...|manga|
|  {[{null, null, La...|84df5379-58d6-4f3...|[{1fccd549-0b48-4...|manga|
|  {[{(Kouroumu 5) F...|84e13f9b-1e9b-4a2...|[{621a6a40-b6b7-4...|manga|
|  {[{Shunkan x Hero...|84e70b38-9586-416...|[{615cdabb-9f3c-4...|manga|
|  {[{null, null, nu...|84ec8c63-a314-458...|[{96ab34b5-8061-4...|manga|
|  {[{Devil☆Clutch, ...|84f5df21-8cc4-44f...|[{6f2b9c03-40c3-4...|manga|
|  {[{The Witch from...|850fd66f-e9cc-444...|[{5e79d2d7-b506-4...|manga|
+----------------------+--------------------+--------

In [54]:
df = (
    spark.read.json(raw.rdd.map(lambda r: r.value))
    .repartition(spark.sparkContext.defaultParallelism * 2)
    .cache()
)

In [83]:
# https://stackoverflow.com/a/70393584

# note that we can't directly do this
# (
#     df.select(
#         "id",
#         F.from_json(
#             F.to_json(F.explode("attributes.altTitles")), schema="map<string, string>"
#         ).alias("col"),
#     )
# ).show()


def convert_via_json(col, type):
    return F.from_json(F.to_json(col), schema=type)


@F.udf(returnType="map<string, string>")
def combine_maps(list):
    if not list:
        return None
    return {k: v for m in list for k, v in m.items()}


def mappify_struct(col):
    return combine_maps(convert_via_json(col, "array<map<string, string>>"))


query_col = "attributes.altTitles"
# but this is valid
alt_titles = (
    df.limit(20)
    .select("id", F.explode(query_col).alias("col"))
    .select("id", convert_via_json("col", "map<string, string>").alias("col"))
    .groupby("id")
    .agg(F.collect_list("col").alias("col"))
    .select("id", combine_maps("col").alias(query_col))
)
alt_titles.show(n=4, truncate=80)

# and so is this

alt_titles = df.select(
    "id",
    mappify_struct(query_col).alias(query_col),
)
alt_titles.show(n=4, truncate=80)

+------------------------------------+-----------------------------------------------------------------+
|                                  id|                                             attributes.altTitles|
+------------------------------------+-----------------------------------------------------------------+
|850fd66f-e9cc-4448-99d3-ae0d3232dffe|   {en -> The Witch from Falling Star Hill, ja -> 星降る丘の魔女}|
|66ed626c-a40c-4fc8-b351-e1a400a9f99c|                             {en -> More Edgy Sasuke's childhood}|
|85cff8c4-a2a8-484b-987d-6a7f781df431|                                             {en -> 百合男子くん}|
|46de95a1-398a-42c7-a35f-4f7370d36cd4|{zh-hk -> 時代大姐大, zh -> 时代大姐大, ja -> いよっおみっちゃん}|
+------------------------------------+-----------------------------------------------------------------+
only showing top 4 rows

+------------------------------------+-----------------------------------------------------------------+
|                                  id|                        

In [87]:
cols_to_mappify = ["altTitles", "description", "links", "title"]
clean = df.select(
    *[c for c in df.columns if c != "attributes"],
    F.struct(
        *[
            F.col(f"attributes.{c}").alias(c)
            for c in df.select("attributes.*").columns
            if c not in cols_to_mappify
        ],
        *[mappify_struct(f"attributes.{c}").alias(c) for c in cols_to_mappify],
    ).alias("attributes"),
)
clean.printSchema()
clean.show(n=4, truncate=80, vertical=True)

root
 |-- id: string (nullable = true)
 |-- relationships: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- related: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)
 |-- attributes: struct (nullable = false)
 |    |-- availableTranslatedLanguages: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- chapterNumbersResetOnNewVolume: boolean (nullable = true)
 |    |-- contentRating: string (nullable = true)
 |    |-- createdAt: string (nullable = true)
 |    |-- isLocked: boolean (nullable = true)
 |    |-- lastChapter: string (nullable = true)
 |    |-- lastVolume: string (nullable = true)
 |    |-- latestUploadedChapter: string (nullable = true)
 |    |-- originalLanguage: string (nullable = true)
 |    |-- publicationDemographic: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- status:

In [89]:
clean.select("attributes.*").show(n=4, truncate=80, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------
 availableTranslatedLanguages   | [en]                                                                             
 chapterNumbersResetOnNewVolume | false                                                                            
 contentRating                  | safe                                                                             
 createdAt                      | 2019-06-25T19:30:58+00:00                                                        
 isLocked                       | false                                                                            
 lastChapter                    |                                                                                  
 lastVolume                     |                                                                                  
 latestUploadedChapter          | 4565fdb7-aa44-40cf-a2b0-719d6cfb644c  